In [6]:
import json
import os
from datetime import datetime, timedelta
from crewai import Agent, Task, Crew, Process
from crewai_tools import CSVSearchTool
from dotenv import load_dotenv, find_dotenv
from langchain.tools import tool
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_openai import ChatOpenAI

In [7]:
load_dotenv(find_dotenv())
llm = ChatOpenAI(model="gpt - 3.5-turbo-0125")

In [9]:
csv_imoveis = CSVSearchTool(csv="files/imoveis.csv")

Inserting batches in chromadb: 100%|██████████| 1/1 [00:05<00:00,  5.04s/it]


In [10]:
#agente corretor de imóveis
corretor_imoveis = Agent(
    role="Corretor de imóveis",
    goal="Obtenha as preferências do cliente e busque imóveis compatíveis no banco de dados.",
    backstory="Especialista no mercado imobiliário, encontre as melhores opções baseadas no perfil do cliente.",
    verbose=True,
    max_iter=5,
    tools=[csv_imoveis],
    allow_delegation=False,
    memory=True)


In [11]:
#tarefa de buscar imóveis
buscar_imoveis = Task(
    description="Pesquise imóveis na região desejada pelo cliente. Considerando faixa de preço e tipo de imóvel.",
    expected_output="Uma lista de imóveis que atendem às preferências do cliente.",
    agent=corretor_imoveis
    )

In [15]:
#dados ficticios de imóveis
from crewai.tools import BaseTool

def obter_precos_imoveis(cidade: str = "geral"):
    precos = {
      "São Paulo": {"Tendência": "aumento", "Percentual":5.2},
      "Rio de Janeiro": {"Tendência": "estável", "Percentual":0.0},
      "Belo Horizonte": {"Tendência": "queda", "Percentual":-3.1},
      "Geral": {"Tendência": "aumento", "Percentual":4.0}
    }
    return precos.get(cidade, precos["Geral"])

In [17]:
class TendenciaPrecosImoveisTool(BaseTool):
    name: str = "Analisador de preços imobiliários"
    description: str = "Obtém tendências de preços de imóveis com base na cidade especificada."

    def _run(self, cidade: str) -> dict:
        """
        Executa a análise de preços imobiliários e retorna a tendência com base na cidade.
        """
        try:
            return obter_precos_imoveis(cidade)
        except Exception as e:
            return {"erro": f"erro ao obter tendência de preços: {str(e)}"}

In [ ]:
analista_mercado = Agent(
    role="Analista de mercado imobiliário",
    goal="Análisa tendências de preços e ajuda a prever a valorização ou desvalorização dos imóveis na cidade {cidade}",
    backstory="Experiente no setor usa dados históricos para prever preços futuros.",
    verbose=True,
    max_iter=5,
    allow_delegation=False,
    memory=True)